In [63]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, PeftModelForCausalLM
import time
import pandas as pd
import random
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu

In [2]:
new_model = "./model/new_model/"
base_model = "./model/base_model/"

my_model = "./llama-Bulgarian-recepies/"

In [3]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=compute_dtype,
    bnb_8bit_use_double_quant=False,
)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    my_model,
    #quantization_config=quant_config,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [00:52<00:00, 26.18s/it]


I face a problem with loading the model and the adapter. When I load the base model Llama 7B via quantization I cannot use PeftModel merge_and_unload. I can't merge LORA layers with an already quantized model.

For now, I am loading the whole Llama model and then I am merging it with the fine-tuned adapter.

In [5]:
bg_model = PeftModel.from_pretrained(model, new_model)
bg_model = bg_model.merge_and_unload()

In [10]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Dec 13 20:50:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0

In [6]:
tokenizer = AutoTokenizer.from_pretrained(my_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [54]:
def generate(text, max_len=200):
  # Ignore warnings
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_len)
  start_time = time.time()
  logging.set_verbosity(logging.CRITICAL)

  # Run text generation pipeline with our next model
  prompt = text
  result = pipe(f"[INST] {prompt} [/INST]")
  end_time = time.time()
  execution_time = end_time - start_time
  print("Execution time:",execution_time)
  #print(result[0]['generated_text'])

  return result[0]['generated_text']

In [55]:
_ = generate("Какво да си сготвя днес? Имам картофи, месо. подправки.")
_ 

Execution time: 8.648853063583374


'[INST] Какво да си сготвя днес? Имам картофи, месо. подправки. [/INST] Начин на приготвяне на Картофи с месо: Картофите се сваряват и се нарязват на филийки. Месото се сварява и се нарязва на малки парчета. В тава се слагат картофите, месото и подправките. Пече се до готовност. 10 минути преди да се извади от фурната се поръсва с настърган кашкавал. 5 минути преди да се извади от фурната се поръсва с магданоз. 2-3 минути преди да се извади от фурната се поръсва'

In [56]:
_ = generate("Знаеш ли как мога да тренирам невронна мрежа?")
_ 

Execution time: 8.872713565826416


'[INST] Знаеш ли как мога да тренирам невронна мрежа? [/INST] Който е увреден от невронните мрежи, може да се смеси с тях. *похлуп* 10 минути и *похлуп* 10 минути. 10 минути *похлуп* и *похлуп*. 10 минути *похлуп* и *похлуп*. 10 минути *похлуп* и *похлуп*. 10 минути *похлуп* и *похлуп*. 10 минути *похлуп* и *похлуп*. 10 минути *похлуп* и *похлуп*. 10 мину'

In [22]:
test_data = pd.read_csv("./data/test_data.csv")

test_data

,promped_text
0,<s>[INST] Искам да сготвя нещо хубаво. Имам на...
1,<s>[INST] Гладен съм. Какво мога да си сготвя ...
2,<s>[INST] Гладен съм. Какво мога да си сготвя ...
3,<s>[INST] Гладен съм. Какво мога да си сготвя ...
4,<s>[INST] Искам да сготвя нещо хубаво. Имам на...
...,...
11159,<s>[INST] Искам да сготвя нещо хубаво. Имам на...
11160,<s>[INST] Искам да сготвя нещо хубаво. Имам на...
11161,<s>[INST] Гладен съм. Какво мога да си сготвя ...
11162,<s>[INST] Искам да сготвя нещо хубаво. Имам на...


In [42]:
# Splitting the 'prompted_text' column
test_data[['instruction', 'response']] = test_data['promped_text'].str.split(r'\s*\[/INST\]\s* ', expand=True)

In [43]:
test_data.instruction = test_data.instruction.str.replace("<s>[INST] ", "")
test_data

,promped_text,instruction,response
0,<s>[INST] Искам да сготвя нещо хубаво. Имам на...,Искам да сготвя нещо хубаво. Имам налични след...,Може да приготвиш Яхния от тиквички с джанки: ...
1,<s>[INST] Гладен съм. Какво мога да си сготвя ...,Гладен съм. Какво мога да си сготвя със: кафяв...,Начин на приготвяне на Ливански пилаф с леща и...
2,<s>[INST] Гладен съм. Какво мога да си сготвя ...,Гладен съм. Какво мога да си сготвя със: л пря...,Начин на приготвяне на Домашен катък: Прясното...
3,<s>[INST] Гладен съм. Какво мога да си сготвя ...,Гладен съм. Какво мога да си сготвя със: порто...,Начин на приготвяне на Чийзкейк крем карамел с...
4,<s>[INST] Искам да сготвя нещо хубаво. Имам на...,Искам да сготвя нещо хубаво. Имам налични след...,Може да приготвиш Шницел с кайма: Картофите се...
...,...,...,...
11159,<s>[INST] Искам да сготвя нещо хубаво. Имам на...,Искам да сготвя нещо хубаво. Имам налични след...,Може да приготвиш Кошнички от бекон с плънка: ...
11160,<s>[INST] Искам да сготвя нещо хубаво. Имам на...,Искам да сготвя нещо хубаво. Имам налични след...,Може да приготвиш Руска салата у дома: Всичко ...
11161,<s>[INST] Гладен съм. Какво мога да си сготвя ...,Гладен съм. Какво мога да си сготвя със: карто...,Начин на приготвяне на Млечни картофи: Картофи...
11162,<s>[INST] Искам да сготвя нещо хубаво. Имам на...,Искам да сготвя нещо хубаво. Имам налични след...,Може да приготвиш Лазаня със зеленчуци и риба ...


In [71]:
def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu

In [64]:
random.seed(42)

random_numbers = random.sample(range(test_data.shape[0]), 10)

for i in random_numbers:
    picked_txt = test_data.instruction[i]
    print("Instruction:")
    print(picked_txt)
    print("The Real answer is: ")
    print(test_data.response[i] + "\n")
    print("The Generated answer is:")

    gen_txt = generate(picked_txt, max_len=500)
    print(gen_txt.split("[/INST] ")[1])

    print("================================================= \n")

Instruction:
Искам да сготвя нещо хубаво. Имам налични следните продукти: яйца, пудра захар, масло, кисело мляко, брашно, бакпулвер ( пакетче), ванилия, кристална захар, сладко
The Real answer is: 
Може да приготвиш Дамски каприз: Белтъците се отделят от жълтъците. Разбиват се жълтъците с пудрата захар, маслото се разтопява и се добавя към получената смес. Добавят се и киселото мляко, брашното, ванилията и бакпулвера. Разбива се добре, изсипва се в тавичка или формички и се пече до порозовяване. Когато основата е готова се намазва със сладко по избор. Белтъците се разбиват със захарта и този крем се намазва върху сладкото. Запича се леко.

The Generated answer is:
Execution time: 47.139244079589844
Може да приготвиш Мъфини с кристална захар и сладко: Яйцата се разбиват със захарта. Добавят се киселото мляко, разтопеното масло и брашното с бакпулвера. Разбърква се добре и се добавя ванилията. 100 г от сместа се използват за сладкото, а останалата се разпределя в 12 купички. Пекат се в п